In [3]:
# ACKNOWLEDGEMENTS:
# Code copied and modified (MIT License) from 
# (https://github.com/JanZrimec/DeepExpression/blob/master/scripts/Chapter_3_1.ipynb)
# Original author: Jan Zrimec (https://github.com/JanZrimec)

import numpy as np
import pandas as pd

from scipy import stats, special, spatial

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import importlib.util
import os

import warnings
warnings.filterwarnings('ignore')

#s.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
## datasets
folder = '../data/'

# Data
fn1 = 'scerevisiae_tpm_full_cut5.csv'
fn2 = 'scerevisiae_accessions.tsv'
fn3 = 'scerevisiae.rsd1.lmbda_22.npz'
fn4 = 'Saccharomyces_cerevisiae.R64-1-1.regions.csv'

fn5 = 'organisms_deep_results.csv'

fname_data1 = folder+'scerevisiae.rsd1.lmbda_22.npz'
fname_module1 = folder+'Model_C3F2all.py'
fname_p1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best.p'
fname_weights1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best'

fn6 = 'df_Keren2013_predictions.csv'
fn7 = 'df_Yamanishi2013_predictions.csv'


In [5]:
def load_data(fname):
    # X is multi-variable array
    # Y contains single variable - fix shape for Keras

    npzfile = np.load(fname,allow_pickle=True)
    Xh_train = npzfile['arr_0']
    Xh_test = npzfile['arr_1']
    Xv_train = npzfile['arr_2'][:,8:]
    Xv_test = npzfile['arr_3'][:,8:]
    Y_train = npzfile['arr_4']
    Y_test = npzfile['arr_5']
    names_train = npzfile['arr_6']
    names_test = npzfile['arr_7']

    X_train = list()
    X_train.append(Xh_train)
    #X_train.append(Xv_train)
    X_test = list()
    X_test.append(Xh_test)
    #X_test.append(Xv_test)

    Y_train = Y_train.astype(np.float32).reshape((-1,))
    Y_test = Y_test.astype(np.float32).reshape((-1,))

    return (np.concatenate((Y_train, Y_test)),
            np.concatenate((names_train,names_test)),
            np.concatenate((Xv_train,Xv_test)))

# load modules - POC model
def load_module(model_path):
    '''loads module containing models given path'''
    spec = importlib.util.spec_from_file_location('module',model_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module


In [6]:
## load data and model

module = load_module(fname_module1)

# load p
p = pd.read_csv(fname_p1,sep=';')
p = p.iloc[0]
p.drop(['path','overfit'],inplace=True)
print(p.shape)

# load data
X_train, X_test, Y_train, Y_test = module.load_data(fname_data1)
# Y = np.concatenate((Y_train, Y_test))
# X = np.concatenate((X_train[0], X_test[0]))
print(len(X_test))
print(len(Y_test))

# model
input_shapes = [sl.shape[1:] for sl in X_train]
model = module.POC_model(input_shapes, p)
# set weights model 1
print("Loading model 1 from disk..")
model.load_weights(fname_weights1)


2024-04-08 19:12:59.575102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(42,)
2
425


2024-04-08 19:13:02.220503: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Loading model 1 from disk..


In [7]:
file_COMBO_npz = "/home/andy/swap/swapped_70225.npz"

working_npz = np.load(file_COMBO_npz, allow_pickle=True)
X_hot = working_npz["arr_0"]
X_var = working_npz["arr_1"]

current_input = list()
current_input.append(X_hot)
current_input.append(X_var)

print("loaded input")

loaded input


In [8]:
Y_pred = model.predict(current_input)
print("predictions done")

2195/2195 [==============================] - 189s 86ms/step
predictions done


In [9]:
npy_out_file = "/home/andy/swap/swapped_PREDICTIONS.npy"
np.save(npy_out_file, Y_pred)
print("saved")

saved


In [10]:
test_Y_pred = np.load(npy_out_file, allow_pickle=True)
print(test_Y_pred)

[[10.795016]
 [10.045161]
 [ 9.695206]
 ...
 [ 9.129368]
 [ 9.307976]
 [ 9.201614]]


In [11]:
print(len(test_Y_pred))

70225
